In [14]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import scipy.stats as st
from IPython.display import display, Markdown

from common_plotting import read_data, plot, years, months, yearsFmt, add_thp
from common_analysis import analyse_dcv, analyse_dcv_absolute, add_dut_and_setting_group, combine_stds_sum

In [4]:
thp_data, data = read_data(['ks3458a-4w-res-log.csv', 'ks3458a-2w-res-log.csv'])
res_2w = data['ks3458a-2w-res-log.csv']
res_4w = data['ks3458a-4w-res-log.csv']

In [18]:
def analyse_ohms(data):
    meter = 'ag3458a_2'
    data_with_groups = add_dut_and_setting_group(data)
    cleaned = data_with_groups.groupby('group').apply(lambda x: x.iloc[1:-1]).droplevel(0)
    data_grouped_by_dut_setting = cleaned.groupby('group').agg({f'{meter}_ohm': ['mean', 'std', 'sem', 'count'], 'temperature': ['mean', 'std', 'sem', 'count'], 'dut': 'last', 'dut_setting': 'last'})
    data_with_dut_group = data_grouped_by_dut_setting.copy()
    data_with_dut_group['dut_group'] = (data_with_dut_group['dut']['last'] != data_with_dut_group['dut']['last'].shift(1)).cumsum()
    data_with_dut_group.columns = ['_'.join(col) for col in data_with_dut_group.columns.values]
    data_grouped_by_dut = data_with_dut_group.groupby('dut_group_').agg({'dut_last': 'last', f'{meter}_ohm_mean': lambda v: np.mean(np.abs(v)), (f'{meter}_ohm_sem'): combine_stds_sum, 'temperature_mean': 'mean'})
    data_grouped_by_dut.columns = ['dut', 'ohm_mean', 'ohm_sem', 'temperature_mean']
    return data_grouped_by_dut

res_4w_results = analyse_ohms(res_4w)
res_2w_results = analyse_ohms(res_4w)

In [19]:
res_4w_results[res_4w_results.dut == 'SR104']

,dut,dcv_mean,dcv_sem,temperature_mean
dut_group_,,,,
7,SR104,10000.104702,0.000782,30.743245
9,SR104,10000.102610,0.000478,32.045664
11,SR104,10000.101163,0.001709,32.165690
20,SR104,10000.048989,0.000218,37.836020
22,SR104,10000.050774,0.000192,37.784630
26,SR104,10000.044596,0.001808,38.316964
29,SR104,10000.026283,0.000513,38.268837
32,SR104,10000.021148,0.000568,38.368661
35,SR104,10000.027913,0.000805,38.130769
